In [155]:
from bs4 import BeautifulSoup
import numpy as np
import requests
import re
import itertools
import random
import pandas as pd

In [156]:
np.random.seed(42)
url="https://www.bindingdb.org/validation_sets/index-1.jsp"
r = requests.get(url)
soup = BeautifulSoup(r.content,'html.parser')
properstruct = re.compile('http://www.rcsb.org/pdb/explore/explore.do')

In [157]:
rows = soup.find_all('tr')
imp_rows = [i for i in rows if len(i.find_all('a')) > 0]

In [158]:
rec_to_pdb = dict()
list_of_pdbs=[]

In [162]:
system = ""
list_rec = []
for idx, i in enumerate(imp_rows):
    name = i.span
    if name is not None:
        if len(list_rec):
            rec_to_pdb[system]=list_rec
            list_rec = []
        system = name.text
        continue
    #non-system names should make it here
    if 'bold' in  i.attrs['class']:
        pdbid = [j.text for j in i.find_all('a',href=properstruct)]
        list_rec+= pdbid
        list_of_pdbs+= pdbid
if len(list_rec):
    rec_to_pdb[system]=list_rec

In [174]:
groups = [*rec_to_pdb]
train = round(0.8*len(groups))
val=round(1.0/3*len(groups))
np.random.shuffle(groups)

In [178]:
train_data = pd.read_csv('training_input.txt', delimiter=' ', header =None)
train_data.columns = ['label','reglabel','og', 'lig1','lig2']
train_data['recs'] = train_data['og'].astype(str).str[:4]
train_data.head()
#train_data['rec'] = pd.Series(train_data.apply(lambda x: re.match(x,rec_of_row)[0],axis=1), index=train_data.index)
pdb_to_ind = dict()
for pdb in list_of_pdbs:
    pdb_to_ind[pdb]=train_data.index[train_data['recs'] == pdb].tolist()
del train_data['recs']

In [179]:
perm1_train=groups[:train]
remainder=groups[train:]
perm1_val = remainder[:len(remainder)//2]
perm1_test= remainder[len(remainder)//2:]
print(len(perm1_train),len(perm1_val),len(perm1_test))
def make_csvs(train_data, groups, name, rec_to_pdb, pdb_to_ind):
    csv_out=pd.DataFrame()
    for val in groups:
        rec =rec_to_pdb[val]
        for r in rec:
            all_rec = train_data.loc[pdb_to_ind[r]]
            csv_out = csv_out.append(all_rec)
    csv_out.to_csv(name+'.txt',sep=' ',header=False,index=False)
    return csv_out.shape[0]


double_check = 0
for grouping,name in zip([perm1_train,perm1_val,perm1_test],['train','val','test']):
    val = make_csvs(train_data, grouping,name,rec_to_pdb, pdb_to_ind)
    print(float(val)/train_data.shape[0])
    double_check += val

111 14 14
0.7860762574962934
0.08949080528447188
0.12443293721923476


In [177]:
print(double_check)
print(train_data.shape[0])

90378
90378


In [92]:
97+42

139

In [121]:
'text'.split()

['text']